Research Links:
* https://www.researchgate.net/publication/258716133_Miniature_Near-Infrared_NIR_Spectrometer_Engine_For_Handheld_Applications

* https://journals.sagepub.com/doi/abs/10.1255/jnirs.1084

In [2]:
#!pip install -q efficientnet

     |████████████████████████████████| 50 kB 4.2 MB/s 


In [3]:
import pandas as pd
import ssl
from sklearn.cluster import KMeans
import numpy as np
import json
import random, string
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
# import sweetviz as sv
tqdm.pandas()
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split,cross_val_score, StratifiedKFold, StratifiedShuffleSplit, GroupKFold
from sklearn.metrics import accuracy_score,confusion_matrix,recall_score,precision_recall_curve, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
# from lightgbm import LGBMClassifier
# import lightgbm as lgb
# from xgboost import XGBClassifier
# from catboost import CatBoostClassifier
import tensorflow as tf
import tensorflow.keras.layers as L

import efficientnet.tfkeras as efn
%matplotlib inline
sns.set_style("whitegrid")
seed = 42

ssl._create_default_https_context = ssl._create_unverified_context


In [ ]:
# Import the dataset
train = pd.read_csv('Data/Updated_Train.csv', index_col=0)
test = pd.read_csv('Data/Updated_Test.csv', index_col=0)
# train_trimmed = pd.read_csv('Data/Updated_Train_trimmed.csv', index_col=0)
# test_trimmed = pd.read_csv("Data/Updated_Test_trimmed.csv", index_col=0)
spectra_hgb = pd.read_csv('Data/Spectra_hgb.csv', index_col=0).reset_index()
spectra_chol = pd.read_csv('Data/Spectra_chl.csv', index_col=0)
spectra_glu = pd.read_csv('Data/Spectra_gluc.csv', index_col=0).reset_index()
spectra_skinfatblood = pd.read_csv('Data/Spectra_skinfatblood.csv', index_col=0)


### **Exploratory Data Analysis**

In [ ]:
train.head()

In [ ]:
test.head()

Removing donation_id 6824 from the train data as the data producer said it is not good.

In [ ]:
dropped_index =  train[train["donation_id"] == 6824].index 
train_d = train.drop(dropped_index)
print(f"Train data shape is:",train_d.shape)
print(f"Test data shape is:",test.shape)

`Reading_ID` is in test but not in train. There are three target columns in the train dataset, `hdl_cholesterol_human`, `hemoglobin(hgb)_human`, and `cholesterol_ldl_human` - Multi target data. 

The data from each samples are repeated about 60 times at different temperatures and humidity which were said to affect the spectrum data. This might cause overfitting if fed into the model. How do I segment this data? 

In [ ]:
print(f"There are",train_d["donation_id"].nunique(), "donation ids in the dataset")

In [ ]:
train_d.describe()

There seems to be no outlier in the dataset. 

In [ ]:
# Check the unique values of the targets
print(train_d["hdl_cholesterol_human"].unique())
print(train_d["hemoglobin(hgb)_human"].unique())
print(train_d["cholesterol_ldl_human"].unique())

This looks uniform, all target variables are classified into `low`, `ok` and `high`.

In [ ]:
# # Trim the train and test data of the noise
# abs_to_drop = ['absorbance0', 'absorbance1', 'absorbance2', 'absorbance3',
#        'absorbance4', 'absorbance5', 'absorbance6', 'absorbance7',
#        'absorbance8', 'absorbance9','absorbance10', 'absorbance159', 'absorbance160', 'absorbance161',
#        'absorbance162', 'absorbance163', 'absorbance164', 'absorbance165',
#        'absorbance166', 'absorbance167','absorbance168', 'absorbance169']
# train_trim = train.drop(columns = abs_to_drop, axis=1)
# test_trim = test.drop(columns = abs_to_drop, axis=1)

In [ ]:
# Check for missing value
print(str(train_d.isnull().sum().sum()) + " missing value in train")
print(str(test.isnull().sum().sum()) + " missing value in test")

In [ ]:
def find_correlated_features(df, threshold, target_variable1, target_variable2, target_variable3):
    # remove target column
    df = df.drop(columns=[target_variable1,target_variable2, target_variable3]).copy()
    # Get correlation matrix
    corr_matrix = df.corr().abs()
    # Take half of the matrix to prevent doubling results
    corr_matrix = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k = 1).astype(bool))
    # Restructure correlation matrix to dataframe
    df = corr_matrix.stack().reset_index()
    df.columns = ['feature1', 'feature2', 'corr_coef']
    # Apply filter and sort coefficients
    df = df[df.corr_coef >= threshold].sort_values('corr_coef', ascending=False)
    return df

In [ ]:
# Correlation analysis
find_correlated_features(train_d,0.7,'hdl_cholesterol_human', 'cholesterol_ldl_human', 'hemoglobin(hgb)_human')

In [ ]:
train = train_trimmed.copy()
test = test_trimmed.copy()
targets = ['hdl_cholesterol_human', 'cholesterol_ldl_human', 'hemoglobin(hgb)_human']
features = [col for col in train.columns if ("absorbance" in col)] + ["temperature", "humidity"]

In [ ]:
train.head()

In [ ]:
report = sv.analyze(train_trimmed)
report.show_html('report.html')

In [ ]:
# #Plotting heatmap
# def half_masked_corr_heatmap(dataframe, title=None, file=None):
#     plt.figure(figsize=(20,10))
#     sns.set(font_scale=1)
#     mask = np.zeros_like(dataframe.corr())
#     mask[np.triu_indices_from(mask)] =  True

#     with sns.axes_style("white"):
#         sns.heatmap(dataframe.corr(), mask=mask, annot=True, cmap="coolwarm")
#     if title:
#         plt.title(f"\n{title}\n", fontsize=18)
#         plt.xlabel("")
#         plt.ylabel("")
#         plt.show()
#     return

# half_masked_corr_heatmap(train_trimmed, "Absorbance correlation heatmap")

In [ ]:
#train.drop(columns = targets, axis = 1)

In [ ]:
# train_df = train_copy[features].copy()
# test_df = test[features].copy()

#### Spectra Data

 -  Spectra Heamoglobin - Oxyhemoglobin

In [ ]:
# oxy_hgb = spectra_hgb[['HbO2', 'Unnamed: 4']]
# oxy_hgb = spectra_hgb.rename(columns = {'Unnamed: 4' : 'absorbances'})
# oxy_hgb = oxy_hgb.iloc[1:,3:5]
# oxy_hgb = oxy_hgb.astype(float)
# # Plot the values
# plt.title("Plot of Oxygenated blood against absorbances")
# sns.lineplot(data = oxy_hgb, x = 'HbO2', y = 'absorbances');

 -  Spectra Heamoglobin - Deoxyhemoglobin

In [ ]:
# deoxy_hgb = spectra_hgb[['Hb', 'Unnamed: 1']]
# deoxy_hgb = spectra_hgb.rename(columns = {'Unnamed: 1' : 'absorbances'}).reset_index()
# deoxy_hgb = deoxy_hgb.iloc[1:,1:3]
# deoxy_hgb = deoxy_hgb.astype(float)
# # Plot the values
# plt.title("Plot of Deoxygenated blood against absorbances")
# sns.lineplot(data = deoxy_hgb, x = 'Hb', y = 'absorbances');

In [ ]:
# deoxy_hgb['Hgb_high_1'] = list(train[train['hemoglobin(hgb)_human'] == 'high'].iloc[0][0:148].values)
# deoxy_hgb['Hgb_high_2'] = list(train[train['hemoglobin(hgb)_human'] == 'high'].iloc[1000][0:148].values)
# deoxy_hgb['Hgb_low_1'] = list(train[train['hemoglobin(hgb)_human'] == 'low'].iloc[0][0:148].values)
# deoxy_hgb['Hgb_low_2'] = list(train[train['hemoglobin(hgb)_human'] == 'low'].iloc[1000][0:148].values)
# deoxy_hgb['Hgb_ok_1'] = list(train[train['hemoglobin(hgb)_human'] == 'ok'].iloc[0][0:148].values)
# deoxy_hgb['Hgb_ok_2'] = list(train[train['hemoglobin(hgb)_human'] == 'ok'].iloc[1000][0:148].values)
# deoxy_hgb.head()

In [ ]:
# plt.title("Plot of Oxygenated blood against heamoglobin high absorbances")
# sns.lineplot(data = deoxy_hgb, x = 'Hb', y = 'Hgb_high_1');

In [ ]:
# plt.title("Plot of Oxygenated blood against heamoglobin high absorbances")
# sns.lineplot(data = deoxy_hgb, x = 'Hb', y = 'Hgb_high_2');

In [ ]:
# plt.title("Plot of Oxygenated blood against heamoglobin low absorbances")
# sns.lineplot(data = deoxy_hgb, x = 'Hb', y = 'Hgb_high_1');

In [ ]:
# plt.title("Plot of Oxygenated blood against heamoglobin low absorbances")
# sns.lineplot(data = deoxy_hgb, x = 'Hb', y = 'Hgb_low_2');

In [ ]:
# plt.title("Plot of Oxygenated blood against heamoglobin ok absorbances")
# sns.lineplot(data = deoxy_hgb, x = 'Hb', y = 'Hgb_ok_1');

In [ ]:
# plt.title("Plot of Oxygenated blood against heamoglobin ok absorbances")
# sns.lineplot(data = deoxy_hgb, x = 'Hb', y = 'Hgb_ok_2');

 -  Spectra Data - Cholesterol

In [ ]:
# spectral_chols = spectra_chol.rename(columns = {'Unnamed: 1' : 'absorbances'}).reset_index()
# spectral_chols = spectral_chols.iloc[1:,0:2]
# spectral_chols = spectral_chols.astype(float)
# # Plot the values
# plt.title("Plot of Cholesterol against absorbances")
# sns.lineplot(data = spectral_chols, x = 'Cholesterol', y = 'absorbances');

In [ ]:
train.columns

In [ ]:
# spectral_chols['chol_high_1'] = list(train[train['hdl_cholesterol_human'] == 'high'].iloc[0][0:170].values)
# spectral_chols['chol_high_2'] = list(train[train['hdl_cholesterol_human'] == 'high'].iloc[1000][0:170].values)
# spectral_chols['chol_low_1'] = list(train[train['hdl_cholesterol_human'] == 'low'].iloc[0][0:170].values)
# spectral_chols['chol_low_2'] = list(train[train['hdl_cholesterol_human'] == 'low'].iloc[1000][0:170].values)
# spectral_chols['chol_ok_1'] = list(train[train['hdl_cholesterol_human'] == 'ok'].iloc[500][0:170].values)
# spectral_chols['chol_ok_2'] = list(train[train['hdl_cholesterol_human'] == 'ok'].iloc[1000][0:170].values)
# spectral_chols.head()

 -  Spectra Heamoglobin - Skin Fat Blood

In [ ]:
# spectral_skin = spectra_skinfatblood.rename(columns = {'Unnamed: 1' : 'absorbances'}).reset_index()
# spectral_skin = spectral_skin.iloc[1:,0:2]
# spectral_skin = spectral_skin.astype(float)
# # Plot the values
# plt.title("Plot of Skin against absorbances")
# sns.lineplot(data = spectral_skin, x = 'Skin', y = 'absorbances');

In [ ]:
# spectral_skin['skin_high_1'] = list(train[train['hdl_cholesterol_human'] == 'high'].iloc[0][0:170].values)
# spectral_skin['skin_high_2'] = list(train[train['hdl_cholesterol_human'] == 'high'].iloc[1000][0:170].values)
# spectral_skin['skin_low_1']= list(train[train['hdl_cholesterol_human'] == 'low'].iloc[0][0:170].values)
# spectral_skin['skin_low_2'] = list(train[train['hdl_cholesterol_human'] == 'low'].iloc[1000][0:170].values)
# spectral_skin['skin_ok_1'] = list(train[train['hdl_cholesterol_human'] == 'ok'].iloc[500][0:170].values)
# spectral_skin['skin_ok_2']= list(train[train['hdl_cholesterol_human'] == 'ok'].iloc[1000][0:170].values)
# spectral_skin.head()

In [ ]:
# spectral_fat = spectra_skinfatblood.rename(columns = {'Unnamed: 4' : 'absorbances'}).reset_index()
# spectral_fat = spectral_fat.iloc[1:,3:5]
# spectral_fat = spectral_fat.astype(float)
# # Plot the values
# plt.title("Plot of Fat against absorbances")
# sns.lineplot(data = spectral_fat, x = 'Fat', y = 'absorbances');

In [ ]:
# spectral_skin['skin_high_1'] = list(train[train['hdl_cholesterol_human'] == 'high'].iloc[0][0:170].values)
# spectral_skin['skin_high_2'] = list(train[train['hdl_cholesterol_human'] == 'high'].iloc[1000][0:170].values)
# spectral_skin['skin_low_1']= list(train[train['hdl_cholesterol_human'] == 'low'].iloc[0][0:170].values)
# spectral_skin['skin_low_2'] = list(train[train['hdl_cholesterol_human'] == 'low'].iloc[1000][0:170].values)
# spectral_skin['skin_ok_1'] = list(train[train['hdl_cholesterol_human'] == 'ok'].iloc[500][0:170].values)
# spectral_skin['skin_ok_2']= list(train[train['hdl_cholesterol_human'] == 'ok'].iloc[1000][0:170].values)

In [ ]:
# spectral_blood = spectra_skinfatblood.rename(columns = {'Unnamed: 7' : 'absorbances'}).reset_index()
# spectral_blood = spectral_blood.iloc[1:,6:8]
# spectral_blood = spectral_blood.astype(float)
# # Plot the values
# plt.title("Plot of Deoxygenated whole blood against absorbances")
# sns.lineplot(data = spectral_blood, x = 'Deoxygenated whole blood', y = 'absorbances');

In [ ]:
# trimmed_range = list(test_trimmed.columns[2:150])

In [ ]:
# wavelengths = list(spectra_glu[1:]['Glucose'])

# absorbance_dict = dict()

# val = 0

# for wave in wavelengths:
    
#     absorbance_dict[f'absorbance{val}'] = wave
    
#     val += 1

In [ ]:
# absorbance_dict

In [ ]:
# #create dictionary to convert absorbance to it's equivalent wavelength range

# trimmed_absorbances = dict()

# val = 11

# for trimmed in trimmed_range:
    
#     trimmed_absorbances[trimmed] = f'absorbance{val}'
    
#     val += 1

In [ ]:
# trimmed_absorbances

In [ ]:
# train['tag'] = 'train'
# test['tag'] = 'test'
# all_data = pd.concat([train, test])

In [ ]:
# deoxy_hgb